In [ ]:
!pip install datasets
import torch
from datasets import Dataset
torch.cuda.is_available()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.4 MB/s eta 0:00:00


True

In [ ]:
!pip install datasets transformers huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.3 MB/s eta 0:00:00


In [ ]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
import pandas as pd
df = pd.read_excel('estval.xlsx')

In [ ]:
df.head()

,a,b,c,d,e
0,ARVAMUS,http://arvamus.postimees.ee/1001520/anvar-samo...,1.0,negatiivne,Enam kui kümme aastat tagasi tegutses huumoris...
1,ARVAMUS,http://arvamus.postimees.ee/1001520/anvar-samo...,2.0,vastuoluline,Neid ridu kirjutades tundub isegi ebaviisakas ...
2,ARVAMUS,http://arvamus.postimees.ee/1001520/anvar-samo...,3.0,positiivne,Isiklikult kohtasin natukegi Kukekese moodi po...
3,ARVAMUS,http://arvamus.postimees.ee/1001520/anvar-samo...,4.0,vastuoluline,"Olen näinud ka, kuidas patrull korrarikkujat t..."
4,ARVAMUS,http://arvamus.postimees.ee/1001520/anvar-samo...,5.0,negatiivne,Kummaline on nüüd äkki lugeda politsei ja sise...


In [ ]:
def convertLabelsToNum(row):
  if (row == 'negatiivne'):
    return 0
  elif (row == 'positiivne'):
    return 2
  else:
    return 1

valid_rows = df.loc[(df['d'] == 'negatiivne') | (df['d'] == 'positiivne') | (df['d'] == 'neutraalne'), ['d','e']]
valid_rows = valid_rows.rename(columns={'d': 'label', 'e': 'text'})
valid_rows['label'] = valid_rows['label'].apply(convertLabelsToNum)

In [ ]:
est_dataset = Dataset.from_pandas(valid_rows)
est_dataset = est_dataset.remove_columns('__index_level_0__')

In [ ]:
est_dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 3522
})

In [ ]:
est_dataset = est_dataset.train_test_split(test_size=0.2)

In [ ]:
small_train_dataset = est_dataset['train']
small_test_dataset = est_dataset['test']

In [ ]:
#from datasets import load_dataset
#imdb = load_dataset("imdb")

In [ ]:
#small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(100))])
#small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(10))])

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('tartuNLP/EstBERT')

In [ ]:
small_test_dataset[0]

{'label': 0,
 'text': 'Psühholoog: eestlane ei leia, et tema eluolu linnavalitsusest sõltuks'}

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512) ## data go miss
 
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2817 [00:00<?, ? examples/s]

Map:   0%|          | 0/705 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('tartuNLP/EstBERT', num_labels=3)

Some weights of the model checkpoint at tartuNLP/EstBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are new

In [ ]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
from transformers import TrainingArguments, Trainer
 
training_args = TrainingArguments(
    output_dir="test",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.537900
1000,0.105800
1500,0.017600


TrainOutput(global_step=1770, training_loss=0.1880805243206563, metrics={'train_runtime': 706.2658, 'train_samples_per_second': 39.886, 'train_steps_per_second': 2.506, 'total_flos': 1905058825937946.0, 'train_loss': 0.1880805243206563, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.6680229902267456,
 'eval_accuracy': 0.7446808510638298,
 'eval_f1': 0.7420628798418107,
 'eval_runtime': 6.5624,
 'eval_samples_per_second': 107.431,
 'eval_steps_per_second': 6.857}

In [ ]:
# https://huggingface.co/blog/sentiment-analysis-python

In [ ]:
model.save_pretrained('bertSentiment')